In [1]:
from pathlib import Path
import pandas as pd

In [2]:
RAW_DIR = Path("data/raw")
OUT_DIR = Path("data/integrated")
OUT_DIR.mkdir(parents=True, exist_ok=True)

food_path = RAW_DIR / "Food_Inspections_raw.csv"
sr_path   = RAW_DIR / "311_Service_Requests_sanitary_raw.csv"

In [3]:
food_usecols = [
    "Inspection ID", "DBA Name", "AKA Name", "License #", "Facility Type",
    "Risk", "Address", "City", "State", "Zip", "Inspection Date",
    "Inspection Type", "Results", "Violations", "Latitude", "Longitude"
]

food = pd.read_csv(
    food_path,
    usecols=lambda c: True if c in food_usecols else False,
    dtype={"Zip": "string"},
    low_memory=False,
)

food = food[food["Zip"].notna()].copy()

print("Food rows after filtering:", len(food))

Food rows after filtering: 135111


In [4]:
sr_usecols = ["SR_NUMBER", "SR_TYPE", "ZIP_CODE", "CREATED_DATE", "LATITUDE", "LONGITUDE"]

sr = pd.read_csv(
    sr_path,
    usecols=lambda c: True if c in sr_usecols else False,
    dtype={"ZIP_CODE": "string"},
    low_memory=False,
)

sr = sr[sr["ZIP_CODE"].notna()].copy()

print("311 rows after filtering:", len(sr))

311 rows after filtering: 829962


In [5]:
sr_counts = (
    sr.groupby("ZIP_CODE", as_index=False)
      .agg(sr_count=("SR_NUMBER", "nunique"))
)

sr_by_type = (
    sr.pivot_table(
        index="ZIP_CODE",
        columns="SR_TYPE",
        values="SR_NUMBER",
        aggfunc="nunique",
        fill_value=0,
    )
    .reset_index()
)

sr_counts_full = sr_counts.merge(sr_by_type, on="ZIP_CODE", how="left")

In [6]:
# Merge 311 counts into food inspections by zip code

merged = food.merge(
    sr_counts,
    left_on="Zip",
    right_on="ZIP_CODE",
    how="left",
)

if "sr_count" in merged.columns:
    merged["sr_count"] = merged["sr_count"].fillna(0).astype("int64")

merged.drop(columns=["ZIP_CODE"], inplace=True)

print("Merged rows:", len(merged))

Merged rows: 135111


In [7]:
row_level_out = OUT_DIR / "food_inspections_with_311_by_zip.csv"
merged.to_csv(row_level_out, index=False)

print(f"Row-level merged file created at: {row_level_out}")

Row-level merged file created at: data/integrated/food_inspections_with_311_by_zip.csv


In [8]:
zip_ranked = sr_counts.sort_values(by="sr_count", ascending=False)
zip_ranked

,ZIP_CODE,sr_count
36,60628,37706
28,60620,33810
37,60629,33060
46,60638,29999
25,60617,29799
...,...,...
35,60627,1
68,60706,1
2,60087,1
70,60803,1
